# <img src="https://github.com/JuliaLang/julia-logo-graphics/raw/master/images/julia-logo-color.png" height="100" /> 


This notebook is a **tutorial for assemblying psychometric tests using the package `ATA.jl`**.

You can either run this notebook in Google Colab, or using Jupyter on your own machine.

# Getting Started with Julia in Colab/Jupyter


If you have `Julia` installed locally, I suggest you to copy the code in this notebook and run it in your machine.
Otherwise, if you prefer to use a hosted Google machine, follow these steps:

1. Work on a copy of this notebook: _File_ > _Save a copy in Drive_ (you will need a Google account). Alternatively, you can download the notebook using _File_ > _Download .ipynb_, then upload it to [Colab](https://colab.research.google.com/).
2. Execute the following cell (click on it and press Ctrl+Enter) to install Julia, IJulia (the Jupyter kernel for Julia) and other packages needed in this tutorial (DataFrames, CSV, Distributions, JLD2, ATA, Psychometrics). Installation takes 2-3 minutes. ATA.jl package now works with `Julia 1.6.0`.
3. After you run the cell (the cell directly below this text), go to Colab's menu bar and select Runtime -> Change type of runtime and select Julia 1.6 in Runtime type from the drop down menu. You can also select your prefered hardware acceleration (defaults to GPU).

* _Note_: If your Colab Runtime gets reset (e.g., due to inactivity), repeat steps 2 and 3.

In [ ]:
%%capture
%%shell
if ! command -v julia 3>&1 > /dev/null
then
    wget -q 'https://julialang-s3.julialang.org/bin/linux/x64/1.6/julia-1.6.0-linux-x86_64.tar.gz' \
        -O /tmp/julia.tar.gz
    tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
    rm /tmp/julia.tar.gz
fi
julia -e 'using Pkg; pkg"add IJulia; precompile;"'
echo 'Done'

## Install Required Packages

In [24]:
using Pkg
Pkg.add("CSV")
Pkg.add("DataFrames")
Pkg.add("JLD2")
Pkg.add("HTTP")
Pkg.add("StatsBase")
Pkg.add(PackageSpec(url="https://github.com/giadasp/Psychometrics.jl", rev="master")) 
Pkg.add(PackageSpec(url="https://github.com/giadasp/ATA.jl", rev="master")) 

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.6/Project.toml`
  No Changes to `~/.julia/environments/v1.6/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.6/Project.toml`
  No Changes to `~/.julia/environments/v1.6/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.6/Project.toml`
  No Changes to `~/.julia/environments/v1.6/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.6/Project.toml`
  No Changes to `~/.julia/environments/v1.6/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.6/Project.toml`
  No Changes to `~/.julia/environments/v1.6/Manifest.toml`
    Updating git-repo `https://github.com/giadasp/Psychometrics.jl`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.6/Project.toml`
  No Changes to `~/.julia/environments/v1.6/Manifest.toml`
    Updating git-repo `https://g

## Collect Required Settings and Data

Before starting to build our automated test assembly (ATA) model, we need to prepare the following information (? = optional):

- 🔧__ATA Generic Settings and/or Specifications__:
 - `n_groups`: Number of test groups. The tests inside a group are parallel with respect to constraints and objectives. Having more than group is useful for assemblying multi-stage test (MST) modules.
 - `groups`: Groups' names;
 - `T`: Number of tests to assemble for each test group.
 - `n_items`: Number of items in the item bank (not grouped by friend sets).
 - `?irt_model`: Item Response Theory (IRT) settings are required for objectives which works on the Test Information Function (TIF), or if some Item Characteristic Function (ICF) must be computed. 1-parameter logistic (`"1PL"`), 2-parameter logistic (`"2PL"`), and 3-parameter logistic (`"3PL"`) are supported. Default is `"1PL"`.
 - `?irt_parametrization`: How ability (t) and difficulty (b) and discrimination (a) are linked in the model. `"at-ab"`, `"at-b"`, `"at+ab"`, `"at+b"` are supported. Default is `"at-ab"`.
 - `?irt_D`: The $D$ constant in IRT models. Default is `1.0`.
 - `?enemy_sets_var`: The names of the columns in the item bank which stores the enemy sets, i.e. the sets of items which cannot be chosen in the same test.
 - `?friend_sets_var`: The names of the columns in the item bank which stores the friend sets. Also called "units", they are the sets of items which must be chosen together.
 - `item_use_min`: A vector of length `n_items` which says in how many tests each item must be selected at least. If friend sets are specified, the maximum among all the minimum item use in the same friend sets is chosen.
  - `item_use_max`: A vector of length `n_items` which says in how many tests each item must be selected at most. If friend sets are specified, the minimum among all the maximum item use in the same friend sets is chosen.
 - `length_min`: For each group, specifies the minimum length of the tests.
 - `length_max`: For each group, specifies the maximum length of the tests.
 - `length_weight`: For the siman solver, it specifies the weight on length constraints. `1.0` is suggested as a starting point. If the solver struggles in finding a solution which satisfies length constraints, increase the weights.
 - `?expected_score_var`: If the item bank already contains the item characteristic function (ICF) computed at one ability point or classical theory expected scores (probability of correct answer), you can provide the name of the columns containing this information using this setting. Only one column (so only one ability point) per group can be provided in this way. 
 - `?expected_score_pts`: Otherwise, if the expected scores or ICFs haven't been computed yet, the ability points in which you want them to be computed. By this setting, the ICFs (or expected scores) can be constrained in more than one ability point.
 - `?expected_score_min`: Required if `expected_score_var` or `expected_score_pts` have been specified. For each group (and for each ability point if `expected_score_pts` have been specified), the lower bound for the average ICFs (or expected score).
 - `?expected_score_max`: For each group (and for each ability point if `expected_score_pts` have been specified), the upper bound for the average ICFs (or expected score).  _Ex: For group 1, the tests mut have an expected score higher than 0.5._
 - `?mean_vars`: Only for siman solver, **still not implemented**. Until this feature is not available, and only if the expected score constraints are not needed, the mean value of a quantitative variable in the item bank can be constrained by providing its name by the `expected_score` settings (using the var, min and max settings).
 - `?mean_vars_min`: Only for siman solver, **still not implemented**. For each test group, the lower bound for the average of a certain quantitative variable provided by the item bank.
 - `?mean_vars`: Only for siman solver, **still not implemented**. For each test group, the upper bound for the average of a certain quantitative variable provided by the item bank.
 - `obj_type`: The objective type. Here the list of supported ATA objectives:
   - `"maximin"`: maximizes the minimum across the tests of the test information function (TIF) at predefined ability points, 
   - `"minimax"`: minimizes the maximum distance across the tests between targets and TIFs,
   - `"cc_maximin"`: only for siman solver, maximizes the minimum across the tests of the $\alpha$ quantile of the TIF (Spaccapanico P. et al. (2021), under review, chance-constrained approach, it requires a `Psychometrics` object with samples of item parameter estimates stored in the `chain` field.),
   - `"soyster_maximin"`: maximizes the minimum across the tests of the mean across item parameters samples corrected by 3 standard deviations of the TIF  (Soyster (1973) approach, it requires a `Psychometrics` object with samples of item parameter estimates stored in the `chain` field),
   - `"de_jong_maximin"`: maximizes the minimum across the tests of the mean across item parameters samples corrected by 1 standard deviations of the TIF (De Jong et al. (2009) approach. It requires a `Psychometrics` object with samples of item parameter estimates stored in the chain field.),
   - `"custom"`: only for siman solver, user customizable objective function,
   - `""`: no objective. 
 - `?obj_points`: Required for each objective which uses the TIF. For each test group, the vector of ability points the TIF is computed at. If more than one ability points is provided, the solver tries to maximize the lowest TIF across points.
  - `?obj_targets`: Only for the MINIMAX model and required for that objective. For each test group and ability point, the targets the TIF must approach.
 - `?obj_aux_int`: For randomized objectives (cc_maximin, de_jong_maximin and soyster_maximin) the number of item parameter samples (i.e. the length of the item parameter estimates chain).
 - `?obj_aux_float`: For the cc_maximin model, the quantile.
 - `?categories`: The categorical variables to summarize in the printed results.

- 🔣__Item Bank__ (required):
  A data frame (`DataFrames.DataFrame` object) which contains the features of the items (item parameters, expected scores, item type, friend sets, enemy sets, and other categorical or quantitative variables).
- 🔢__Overlap Matrix__ (optional):
  A $sum(T) \times sum(T)$ matrix containing the maximum overlap between test forms.
- 🔣__Categorical and Quantitative Constraints__ (optional):
  A data frame (`DataFrames.DataFrame` object) with specific columns (group, var, value, min, max, and weight) which contains the upper and lower bounds of the sum of quantitative variables and on number of items presenting a specific value of a categorical variable. For the siman solver, also the weight for each constraint can be specified using the column "weight".
- 📝__A license for one commercial solvers__ (optional 😎):
  If the `JuMP` package is used to solve the ATA model, several open-source and commercial solvers are available. If, for example, the CPLEX solver is selected, a valid license and the `CPLEX.jl` package must be installed and working in the machine in which this notebook is run. In this tutorial, both the pure Julia ATA solver `siman` and the open-source `Cbc` mixed-integer linear programming (MILP) solver are used to solve the same model.



## Build your First ATA Model

Once all the required model specifications and item features have been collected, we are ready to build our ATA model.

In this example a classical MAXIMIN model with test content and structure constraints is solved.
By the MAXIMIN model, the solver seeks the combination of items which produce the maximum TIF (inverse of the expected test measurement precision).
If more than one test must be assembled, the solver tries to maximize the lower TIF across tests.

We want to assemble 30 test forms divided into 3 groups of equal size (10 tests per group).
This means that the 10 tests belonging to a certain group share the same content features and they are parallel with respect to the psychometric properties (TIF, ICFs/expected score).

The IRT model underlying the item parameters is the "1PL", so the difficulty parameter "b" is provided as a column in the item bank.
The items can belong to friend sets as indicated in the column "UNIT".
On the other hand, the column "ENEMY_SET" contains the names of the enemy sets.
The expected score of each item is provided as the column "PROP_CORR".



### ATA Settings and Item Bank

To initialize the an ATA model, the function `start_ata()` must be called.
The `start_ata()` function has the following arguments:
 - (method 1)`settings`: an `InputSettings` object containing the ATA settings and generic specifications\
 alternatively:
 - (method 2)`settings_file`: the name of a file containing an `InputSettings` object assigned to a variable named `Inputs`.


 - (method 1)`bank`: a data frame 
 alternatively:
 - (method 2)`bank_file` and `bank_delim`, the name of a csv file containing the item bank data and the delimitator of the values, respectively.

The item bank can be passed to the ATA model in a separate step using the function `add_bank!(ata_model)` which has the arguments `bank`, `bank_file`, and `bank_delim` as well. 

Let's use the first method for the settings and the second method for the item bank:

In [25]:
using ATA
settings = ATA.InputSettings(
  n_groups = 3,
  groups = ["A", "B", "C"],
  T = [10,10,10],
  n_items = 366,
  irt_model = "1PL",
  irt_parametrization = "at-ab",
  irt_D = 1.0,
  enemy_sets_var = ["ENEMY_SET"],
  friend_sets_var = ["UNIT"],
  item_use_min = fill(0, 366),
  item_use_max = fill(8, 366),
  length_min = [36, 36, 36],
  length_max = [40, 40, 40],
  length_weight = [1.0, 1.0, 1.0],
  expected_score_var = ["PROP_CORR", "PROP_CORR", "PROP_CORR"],
  expected_score_pts =  [[0.0], [0.0], [0.0]],
  expected_score_min = [[0.50], [0.50], [0.50]],
  expected_score_max = [[0.57], [0.57], [0.57]],
  obj_type = "maximin", 
  obj_points = [[-0.60], [0.30], [0.60]],
  categories = ["UNIT", "CAT_1", "CAT_2", "CAT_3", "CAT_4", "CAT_5_6", "CAT_5", "CAT_6"]
);

We download the item bank file from the examples folder of the ATA.jl repository


In [26]:
using HTTP
bank =  HTTP.request("GET","https://raw.githubusercontent.com/giadasp/ATA.jl/master/examples/data/bank.csv")
write("bank.csv", bank.body)

21390

(Method 1) Otherwise, you can load the data frame by using the CSV package and using the Statistics package we can easily summarize the variables.

In [27]:
using CSV
using DataFrames
using StatsBase
bank = CSV.read("bank.csv", delim=';', DataFrames.DataFrame)
println(describe(bank))
println("Frequencies of CAT_1")
println(countmap(bank[!,:CAT_1]))
println("Frequencies of CAT_2")
println(countmap(bank[!,:CAT_2]))
println("Frequencies of CAT_3")
println(countmap(bank[!,:CAT_3]))
println("Frequencies of CAT_4")
println(countmap(bank[!,:CAT_4]))

13×7 DataFrame
 Row │ variable   mean       min      median  max      nmissing  eltype                 
     │ Symbol     Union…     Any      Union…  Any      Int64     Type                   
─────┼──────────────────────────────────────────────────────────────────────────────────
   1 │ ITEM_ID    183.5      1        183.5   366             0  Int64
   2 │ UNIT_ALL              U_1              U_99            0  String
   3 │ UNIT                  U_116            U_86          186  Union{Missing, String}
   4 │ ENEMY_SET             ES_1             ES_9          191  Union{Missing, String}
   5 │ CAT_1                 D_1              D_4             0  String
   6 │ CAT_2                 CAT_2_1          CAT_2_3         0  String
   7 │ CAT_3                 CAT_3_1          CAT_3_5         0  String
   8 │ CAT_4      0.204918   0        0.0     1               0  Int64
   9 │ CAT_5_6               A_BC             S_B             0  String
  10 │ CAT_5                 A          

We are ready to initialize the model.

In [28]:
ata_model = start_ata(settings = settings, bank_file = "bank.csv", bank_delim = ";")

ATA.MaximinModel(ATA.Settings(366, 266, 366×14 DataFrame
 Row │ ITEM_ID  UNIT_ALL  UNIT     ENEMY_SET  CAT_1   CAT_2    CAT_3    CAT_4  ⋯
     │ Int64    String    String?  String?    String  String   String   Int64  ⋯
─────┼──────────────────────────────────────────────────────────────────────────
   1 │       1  U_254     U_254    ES_37      D_3     CAT_2_1  CAT_3_2      0  ⋯
   2 │       2  U_254     U_254    missing    D_3     CAT_2_1  CAT_3_2      0
   3 │       3  U_254     U_254    missing    D_3     CAT_2_1  CAT_3_2      0
   4 │       4  U_256     missing  missing    D_2     CAT_2_1  CAT_3_5      0
   5 │       5  U_257     missing  missing    D_3     CAT_2_1  CAT_3_4      0  ⋯
   6 │       6  U_258     missing  missing    D_4     CAT_2_1  CAT_3_5      0
   7 │       7  U_259     missing  ES_5       D_2     CAT_2_1  CAT_3_5      0
   8 │       8  U_260     U_260    ES_17      D_3     CAT_2_2  CAT_3_2      0
   9 │       9  U_260     U_260    missing    D_3     CAT_2_2  CAT_3_5

### Load the Categorical Constraints and Constraints on Sum of Quantitative Variables 

The number of items having a certain content feature (categorical variable), e.g. CAT_1, CAT_2, and CAT_3 in our item bank, can be constrained to be in a limited interval for each test group.
Moreover, sometimes, it can be helpful to limit the total number of words displayed in a test form.
This value is equal to the sum of the words (quantitative variable) in the items selected to be in the test form.
For example, CAT_4 is a quantitative variable taking the values 0 and 1.

To add these kinds of specifications we need to use a data frame with specific columns.
Thus, we first create an empty data frame with the desired structure and then we populate it with some examples of constraints.
At the end, we will download the data frame used in the examples of ATA.jl repository and we will add it to the model with the `add_constraints!` function.


In [29]:
# Create an empty constraints dataframe
constraints_example = DataFrame(
  group = Int64[],
  var = String[],
  value = Union{Missing,String}[],
  min = Int64[],
  max = Int64[],
  weight = Float64[]
)
# Populate it with a constraint on CAT_1 on the tests in group 1
push!(constraints_example, [1, "CAT_1", "D_3", 10, 15, 1.0])
# Populate it with a constraint on CAT_4 on the tests in group 2 (value is missing because CAT_4 is a quantitative variable)
push!(constraints_example, [2, "CAT_4", missing, 2, 10, 1.0])
println("Constraints example")
println(constraints_example)

# Download dataframe from the repo
constraints =  HTTP.request("GET","https://raw.githubusercontent.com/giadasp/ATA.jl/master/examples/Constraints.csv")
write("constraints.csv", constraints.body)
constraints = CSV.read("constraints.csv", delim = ';', DataFrames.DataFrame);
println("Constraints")
println(constraints)

# Add the constraints to the model (Method 1)
add_constraints!(ata_model, constraints = constraints)
# or Method 2 (do not run)
# add_constraints!(ata_model, constraints_file = "constraints.csv", constraints_delim = ";")


Constraints example
2×6 DataFrame
 Row │ group  var     value    min    max    weight  
     │ Int64  String  String?  Int64  Int64  Float64 
─────┼───────────────────────────────────────────────
   1 │     1  CAT_1   D_3         10     15      1.0
   2 │     2  CAT_4   missing      2     10      1.0
Constraints
32×6 DataFrame
 Row │ group  var      value    min      max      weight  
     │ Int64  String   String   Int64?   Int64?   Float64 
─────┼────────────────────────────────────────────────────
   1 │     1  CAT_1    D_1            6       11      1.0
   2 │     1  CAT_1    D_2            5        9      1.0
   3 │     1  CAT_1    D_3           10       20      1.0
   4 │     1  CAT_1    D_4            4        8      1.0
   5 │     2  CAT_1    D_1            6       11      1.0
   6 │     2  CAT_1    D_2            3        7      1.0
   7 │     2  CAT_1    D_3           10       20      1.0
   8 │     2  CAT_1    D_4            3        7      1.0
   9 │     3  CAT_1    D_1    

### Load the Overlap Matrix

Like the item bank and the constraints data frame, we can add the constraints on the maximum number of overlapping items by using the two methods (matrix vs file name).
We create a matrix of size $30 \times 30$ with values equal to 14.

In [30]:
# Cretate the overlap matrix
overlap = 14 .* ones(30, 30)

# Add the overlap constraints to the model (Method 1)
add_overlap!(ata_model, overlap = overlap)

# or Method 2 (do not run)
# add_overlap!(ata_model, overlap_file = "overlap.csv", overlap_delim)

### Add the Objective Function

Finally, we compute the item information functions and we add the MAXIMIN TIF objective function to the model. 

In [31]:
add_obj_fun!(ata_model)

## Assemble the Model

Let's assemble the model. We choose the pure Julia siman solver. The description of the hyperparameters is given just after each keyword.

In [36]:
assemble!(
  ata_model,
  solver = "siman",
  start_temp = 0.01;
  # Default: `0.1`. Values:  `[0, Inf]`. 
  # Starting temperature, set to minimum for short journeys (if 0 worse solutions will never be accepted).

  geom_temp = 0.1,
  # Default: `0.1`. Values:  `[0, Inf)`.
  # Decreasing geometric factor.

  n_item_sample = Inf,
  # Default: 1. Values: `[1, Inf]`. 
  # Number of items to alter. Set to minimum for a shallow analysis, 
  # set to maximum for a deep analysis of the neighbourhoods.

  n_test_sample = Inf,
  # Default: 1. Values: `[1, Inf]`. 
  # Number of tests to alter. Set to minimum for a shallow analysis, set to maximum for a deep analysis of the neighbourhoods.

  opt_feas = 0.9,
  # Default: 0.0. Values: `[0, Inf)`. 
  # Optimality/feasibility balancer, if 0 only feasibility of solution is analysed. Viceversa, if 1, only optimality is considered (uncontrained model). All the other values are accepted but produce uninterpretable results.

  n_fill = 1,
  # Default: 1. Values: `[0, Inf)`.
  # Number of fill-up phases, usually 1 is sufficient, if start_temp is high it can be higher. 
  # If a starting_design is supplied, it should be set to 0.

  verbosity = 1,
  # Default: 2. Values: `1` (minimal), `2` (detailed).
  # Verbosity level. In the console '+' stands for improvement, '_' for accepting worse solution.
  # The dots are the fill-up improvement steps.

  #! Termination criteria: 

  max_time = 500.0,
  # Default: `1000.0`. Values: `[0, Inf)`.
  # Time limit in seconds.

  max_conv = 10,
  # Default: `2`. Values: `[1, Inf)`. 
  # Maximum convergence, stop when, after max_conv rounds no improvements have been found. 
  # Set to minimum for shallow analysis, increase it for deep analysis of neighbourhoods.

  feas_nh = 1,
  # Default: `0`. Values: `[1, Inf)`. 
  # Maximum number of Feasibility neighbourhoods to explore, set to the minimum if the model is small or not highly constrained.

  opt_nh = Inf
  # Default: `5`. Values: `[1, Inf)`. 
  # Maximum number of Optimality neighbourhoods to explore, set to the minimum if the model is highly constrained.
)

Starting solution: 

  f : 183.000
  infeas :	[  63.0  63.0  63.0  63.0  63.0  63.0  63.0  63.0  63.0  63.0  60.0  60.0  60.0  60.0  60.0  60.0  60.0  60.0  60.0  60.0  60.0  60.0  60.0  60.0  60.0  60.0  60.0  60.0  60.0  60.0    ]
  overlaps :	[    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    ]
  item use :	[    0.0    ]
Fill-up starting...
.................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................-Test 5 f

LoadError: ignored

## Print Results

As can be seen in the printed output, the solver didn't return a feasible solution (Feasibility > 0).
We can have an indepth look at the assembed tests using the function `print_results`.
It creates a file named "results" in the `results_folder`.

In [34]:
print_results(ata_model; results_folder = "results")